# Introduction



This notebook is a template for applying a PPO model in Webots continuous environment.

---
**NOTE**

To use this notebook, please first follow `UseGuide.md` to install the neccessary software and packages.

---

In [7]:
%%capture output 
# captures ALL output in cell to disable tensorflow warnings

import sys
sys.path.insert(0,'../backend')

import numpy as np

import webotsgym as wg
from stable_baselines import PPO1

# Setup World Parameters and Start the Webotsgym


You can setup the Webots environment parameters:

* `config.world_size` , setup the size of Webots environments for training. For example: `config.world_size = 8` will setup a square map of size 8x8 in Webots.
* `config.num_obstacles`, setup the number of obstacles. Each obstacle is a block of size 1x1.
* `config.sim_mode`, used to setup the speed of the simulation of Webots. 
`config.sim_mode = wg.config.SimSpeedMode.NORMAL`, run the simulation in the Real-Time mode.
`config.sim_mode = wg.config.SimSpeedMode.RUN`, run the simulation as fast as possible using all the available CPU power. 
`config.sim_mode = wg.config.SimSpeedMode.FAST`, run the simulation as fast as possible without the graphics rendering, hence the 3d window is black.

In [8]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.RUN

env = wg.WbtGym(config=config)

Accepting on Port:  10201


# Load a Model

Load a model for application. You need to train your own model in `model_training_continuous_world.ipynb` first.

In [9]:
%%capture output 
# captures ALL output in cell to disable tensorflow warnings

model_name = "PPO_webots_continuous"
model = PPO1.load("model/continuous/{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Apply Model on a Random World

Randomly reset the start point, target point, and the obstacles in the environment and apply the loaded PPO model to this environment. Please set `step_limit` to limit the steps for the robot.

You can see how the robot goes to the target. If the robot reaches the target, it will print "Reach the target, congratulations!". If the robot doesn't reach the target in 2000 time steps, it will stop. Then you can rerun this cell to reset the environment.

In [10]:
obs = env.reset()
step_limit = 2000
for i in range(step_limit):
    (action, _states) = model.predict(obs)
    (obs, rewards, done, info) = env.step(action)

    if env.get_target_distance() < 0.1:
        print ('Congratulations! The robot reached the target.')
        break
    if i == step_limit - 1:
        print ('Reached step limit without reaching target.')